<a href="https://colab.research.google.com/github/eaishwa/sql-twitter-analysis/blob/master/SQL_Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Graph Analysis


Many graph analysis compute network centrality, density, shortest paths, and other path-based statistics about a graph.  It may seem that writing a one-off Python script is a good way to perform this analysis, but it turns out that SQL is pretty great at doing this type of analysis!  

## Background of the Data
In this project, I have done a graph analysis to analyze Tweets from [**Twitter Elections Integrity Dataset**](https://about.twitter.com/en_us/values/elections-integrity.html#data).

- Justice Department charged 13 Russian nationals with interfering in American electoral and political processes. The defendants worked for a well-funded “troll factory” called the Internet Research Agency(**IRA**).
- IRA ran a campaign to sow disinformation and discord into American politics via social media (mostly twitter).
- Dataset includes information from 3613 accounts believed to be connected to the Russian Internet Research Agency.

#### Twitter IRA dataset

In reality, the twitter dataset contains the following attributes.

```
tweetid                   # tweet id
userid                    # user id (hashed for users which had fewer than 5,000 followers) 
user_display_name         # name of user (same as userid for anonymized users)
user_screen_name          # the Twitter handle of the user
user_reported_location    # self-reported location
……
```

Please check [Twitter Elections Integrity Datasets Readme](https://storage.googleapis.com/twitter-election-integrity/hashed/Twitter_Elections_Integrity_Datasets_hashed_README.txt) for full description

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Test settings and check the IRA dataset

In [0]:
# set the max display column
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [0]:
%%bigquery --project project-2-db
SELECT *
FROM `project2-236400.twitter.IRA` 
LIMIT 5

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,776360750604492800,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,2,2,3,9,[],[],,
1,300682957591564290,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,0,0,,,[368224442],
2,145886722834710529,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,0,0,,,[342321377],
3,733924993210408960,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,0,0,36,65,[],[],,
4,755811790764802049,201334945,Андрей Манзолевский,Manzal_,Russia,"Блогер, публицист, гражданин, начальник",http://t.co/lsUS2Frfjj,23245,3298,2010-10-11,...,,,2,1,42,137,[],[http://manzal.livejournal.com/425928.html],,


## Step 1

Find the `id` and `text` of Tweets that contain  "MakeAmericaGreatAgain" **and** "Trump" (both **case-insensitive**) .  

For example:

* "#VoteTrump and lets all help #**Trump** #**makeamericagreatagain**" is a match
* "#**Trump** This was our moment. Together, we will **make America great again**!" is not a match


In [0]:
%%bigquery --project project-2-db

SELECT tweetid as id, tweet_text as text
FROM `project2-236400.twitter.IRA` 
WHERE LOWER(tweet_text)
LIKE LOWER('%MakeAmericaGreatAgain%')
AND LOWER(tweet_text)
LIKE LOWER('%Trump%')


,id,text
0,632961738191695872,#DonaldTrump would not rip up the Iran deal #...
1,709598519632404480,Are you ready for #AmericanRenaissance ? RT if...
2,796071368043413504,IT'S DEFINITELY OBVIOUS.... #VoteTrump #EricT...
3,895678567207628800,RT EricTrump: #MakeAmericaGreatAgain!!! 🇺🇸🇺🇸 h...
4,796125831152148480,ΚΑΙ ΠΟΛΥ ΣΑΣ ΕΙΝΑΙ #MakeAmericaGreatAgain http...
5,796118457905532929,#Election2016 #TrumpPence16 #HillaryForPrison2...
6,877843648775430149,🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 President Trump will ...
7,876016634825191424,🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 President Trump will ...
8,874974723335233537,🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 President Trump will ...
9,876424301934768129,🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 President Trump will ...


## Step 2
"**MakeAmericaGreatAgain**" (often abbreviated as MAGA) is a campaign slogan used in American politics that was popularized by Donald Trump in his successful 2016 presidential campaign.

Let's Find out Top 5 month gets the maximum mention of this slogan
"MakeAmericaGreatAgain"(**case-insensitive**) in `tweet_text` .

In [0]:
%%bigquery --project project-2-db

WITH temp AS (
SELECT EXTRACT(YEAR FROM tweet_time) AS year, EXTRACT(MONTH FROM tweet_time) as month, tweetid
FROM `project2-236400.twitter.IRA`
WHERE LOWER(tweet_text)
LIKE LOWER('%MakeAmericaGreatAgain%')
AND LOWER(tweet_text)
LIKE LOWER('%Trump%')
)
SELECT year, month, COUNT(tweetid) AS num
FROM temp
GROUP BY year, month
ORDER BY num DESC
LIMIT 5


,year,month,num
0,2017,6,1637
1,2016,11,341
2,2015,8,151
3,2016,2,149
4,2016,12,136


## Step 3   
**Let's construct the Graph!**

Tweets and retweets can be used to construct the graph where each row is an edge between `userid` and `retweet_userid`.A table “Graph” is created with column names src and dst which stores the edge list of the graph. Only the distinct edges in the table are stored. One user might retweet another user's tweet more than one time. In this case, edge is saved only once (i.e. only one row in the graph table).

The resulting table contains the following columns:
- src (userid)
- dst (retweet_userid)


In [0]:
%%bigquery --project project-2-db

CREATE OR REPLACE TABLE dataset.Graph AS
(
SELECT DISTINCT userid as src, retweet_userid as dst
from `project2-236400.twitter.IRA`
WHERE userid IS NOT NULL
AND userid <> ''
AND retweet_userid IS NOT NULL
AND retweet_userid <> ''
)


""


## Step 4
The indegree of a node in a directed graph is defined as the number of edges which are incoming on the node. Similarly, the outdegree of a node in a directed graph is defined as the number of edges which are outgoing from the node. For more information, you can read - [Indegree and Outdegree](https://en.wikipedia.org/wiki/Directed_graph#Indegree_and_outdegree)

Using this information, let us find out from the GRAPH table which user has the highest indegree and which user has the highest outdegree.

The output contains the columns:
- max_indegree (userid)
- max_outdegree (userid)

In [0]:
%%bigquery --project project-2-db

WITH temp1 AS
(
SELECT src AS max_outdegree
FROM dataset.Graph 
GROUP BY src
ORDER BY COUNT(src) DESC 
LIMIT 1
),
temp2 AS
(
SELECT dst AS max_indegree
FROM dataset.Graph 
GROUP BY dst
ORDER BY COUNT(dst) DESC 
LIMIT 1
)

SELECT b.max_indegree, a.max_outdegree
from temp1 a, temp2 b


,max_indegree,max_outdegree
0,2572896396,a95a911dd6ae864c48ed062cdbe75e5c28dbe0cf57c6db...


## Step 5
Let us define 4 categories of Twitter users in IRA dataset. We will use the average number of likes a user gets on his/her tweets as the first metric and the number of users retweet him (i.e. indegree) as the second metric. Then we can classify each user as follows:
- High indegree, high average number of likes 
- High indegree, low average number of likes
- Low indegree, high average number of likes
- Low indegree, low average number of likes

We will refer to the 'low indegree, low average likes' category of users as "unpopular" users and 'high indegree, high average likes' category of users as "popular" users. 

We define the indegree and average number of likes to be high or low based on the rules below:
   
   1) If `indegree < avg(indegree of all users)` in the graph then indegree is said to be low for the user, else it is considered high.
    
   2) If `avg(likes of all tweets for the user) < avg(likes for all tweets)` in the graph, then the average number of likes is said to be low for the user, else it is considered high.

Now, let us find the conditional probability that given **all tweets from unpopular users**, what is the probability that **they are retweeted from popular users**. We only consider user in this dataset.

The final output contains the column:
- popular_unpopular (conditional probability)

In [0]:
%%bigquery --project project-2-db

-- create table for indegs_unpop
create or replace table dataset.indegs_unpop as
(
with temp as
(
select dst, count(dst) as indegs
from dataset.Graph
group by dst
)
select distinct dst as userid
from temp
where indegs < (select avg(indegs) from temp)
)


""


In [0]:
%%bigquery --project project-2-db

create or replace table dataset.likes_unpop as
(
with temp as
(
select userid, avg(like_count) as avg_likes
from `project2-236400.twitter.IRA`
group by userid
)
select distinct userid
from temp
where avg_likes < (select avg(like_count) from `project2-236400.twitter.IRA`)
)



""


In [0]:
%%bigquery --project project-2-db

create or replace table dataset.unpop as
(
select distinct u1.userid 
from dataset.likes_unpop u1, dataset.indegs_unpop u2
where u1.userid = u2.userid
)

""


In [0]:
%%bigquery --project project-2-db

select count(*) from dataset.pop

,f0_
0,54


In [0]:
%%bigquery --project project-2-db

-- create table for indegs_pop
create or replace table dataset.indegs_pop as
(
with temp as
(
select dst, count(dst) as indegs
from dataset.Graph
group by dst
)
select distinct dst as userid
from temp
where indegs > (select avg(indegs) from temp)
)


""


In [0]:
%%bigquery --project project-2-db

create or replace table dataset.likes_pop as
(
with temp as
(
select userid, avg(like_count) as avg_likes
from `project2-236400.twitter.IRA`
group by userid
)
select distinct userid
from temp
where avg_likes > (select avg(like_count) from `project2-236400.twitter.IRA`)
)



""


In [0]:
%%bigquery --project project-2-db

create or replace table dataset.pop as
(
select distinct u1.userid 
from dataset.likes_pop u1, dataset.indegs_pop u2
where u1.userid = u2.userid
)

""


In [0]:
%%bigquery --project project-2-db

create or replace table dataset.unpoptweets as
(
select unpop.userid, t.tweetid
from dataset.unpop unpop, `project2-236400.twitter.IRA` t
where unpop.userid = t.userid
)

""


In [0]:
%%bigquery --project project-2-db

create or replace table dataset.popretweets as
(
select pop.userid, t.retweet_tweetid
from dataset.pop pop, `project2-236400.twitter.IRA` t
where pop.userid = t.userid
)

""


In [0]:
%%bigquery --project project-2-db

select count(unpop.userid)/(select count(*) from dataset.unpoptweets) as popular_unpopular
from dataset.unpoptweets as unpop, dataset.popretweets as pop
where unpop.tweetid = pop.retweet_tweetid

,popular_unpopular
0,0.000107


In [0]:
%%bigquery --project project-2-db

select (select count(*) from dataset.unpoptweets) as popular_unpopular
from dataset.unpoptweets as unpop, dataset.popretweets as pop
where unpop.tweetid = pop.retweet_tweetid


## Step 6
Given a graph G = (V, E), a “triangle” is a set of three vertices that are mutually adjacent in G i.e. given 3 nodes of a graph A, B, C there exist edges A->B, B->C and C->A which form a triangle in the graph. From the graph table created above, let us find out the number of triangles in the graph.

The output contains the columns:
- no_of_triangles



In [0]:
%%bigquery --project project-2-db

select count(*)/3 as no_of_triangles
from dataset.Graph a, dataset.Graph b, dataset.Graph c
where a.dst = b.src 
and b.dst = c.src
and a.src = c.dst
and a.src != a.dst
and b.src != b.dst
and c.src != c.dst


,no_of_triangles
0,1516773.0


## Step 7
Let's check who has the most impact in IRA dataset.

The **PageRank algorithm** is used to rank the importance of nodes in a graph. It works by counting the number of edges incident to a node to determine how important the node is. The underlying assumption is that more important nodes are likely to receive more links from other nodes. Find the top 100 nodes with the highest PageRank in the graph.
It is not possible to use "WITH RECURSIVE" on BigQuery. So let us develop an iterative implementation for PageRank (like the BFS example mentioned below).

The algorithm is run for 20 iterations and the output table contains the following columns:
- twitter_username (the twitter_username of the user)
- page_rank_score



---


This algorithm works as follows - Assume a small universe of four web pages: A, B, C and D. PageRank is initialized to the same value for all pages since we assume a probability distribution between 0 and 1 as the PageRank for each node. Hence the initial value for each page in this example is 0.25. If the only links in the system were from pages B->A, C->A and D->A, each link would transfer 0.25 PageRank to A upon the next iteration, for a total of 0.75 i.e. PR(A) = PR(B) + PR(C) + PR(D). 

Now, suppose instead that we have the links B->C, B->A, C->A, D->A, D->B, D->C. Thus, upon the first iteration, page B would transfer half of its existing value, or 0.125, to page A and the other half, or 0.125, to page C. Page C would transfer all of its existing value, 0.25, to the only page it links to, A. Since D had three outbound links, it would transfer one third of its existing value, or approximately 0.083, to A. At the completion of this iteration, page A will have a PageRank of approximately 0.458.
PR(A)=PR(B)/2 + PR(C)/1 + PR(D)/3.

Thus, we can write the PageRank of A as:
PR(A)= PR(B)/L(B) + PR(C)/L(C) + PR(D)/L(D) where L(x) gives us the number of outbound links for any node x. 

In general, the PageRank value for a page u is dependent on the PageRank values for each page v contained in the set containing all pages linking to page u, divided by the number of links from page v. 
It is given by the formula: ![](https://www.geeksforgeeks.org/wp-content/ql-cache/quicklatex.com-aafd3a0d9f8bb8325cf2b41a4a839bbf_l3.svg)


In [0]:
project_id = 'project-2-db'

In [0]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

def pagerank(client, n_iter):
  
  q1 = """
  
  create or replace table dataset.pagerank as 
  (
  select distinct userid, (select 1/count(distinct userid) from `project2-236400.twitter.IRA`) as rank
  from `project2-236400.twitter.IRA` 
  )
  
  """
  
  q2 = """
  
  create or replace table dataset.outdegree as
  (
  select src as userid, count(src) as outdegree
  from dataset.Graph
  group by src
  )
  
  """
  
  job = client.query(q1)
  results = job.result()
  job = client.query(q2)
  results = job.result()
  
  for i in range(0,n_iter):
    print("iteration: " + str(i))
    
    q3 = """
    
    create or replace table dataset.pagerank as
    (
    select g.dst as userid, sum(pr.rank/o.outdegree) as rank
    from dataset.pagerank pr, dataset.outdegree o, dataset.Graph g
    where pr.userid = o.userid and g.src = pr.userid
    group by g.dst
    )
    
    """
    job = client.query(q3)
    results = job.result()
   
  q4 = """
  
  select distinct p.user_display_name as twitter_username, pr.rank
  from `project2-236400.twitter.IRA` p, dataset.pagerank pr
  where p.userid = pr.userid
  order by pr.rank desc
  limit 100
  
  """
  
  job = client.query(q4)
  display(job.to_dataframe())
  
  

In [0]:
pagerank(client,20)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19


,twitter_username,rank
0,4066d5682033750329f2ccae4e6bb049128ea4ab2273ec...,2.727025e-04
1,dd3e6a1f6e52a64ace2b64d92f8612456225be9bc6ecf7...,2.727025e-04
2,f53718cfb6d0333d9d89b8af297735e32410a8823f38dd...,2.727025e-04
3,Вестник Крыма,4.515136e-05
4,Вестник Петербурга,4.506164e-05
5,Вестник Тулы,4.499626e-05
6,Вестник Новосибирска,4.470186e-05
7,Вестник Челябинска,4.464964e-05
8,Вестник Ростова,4.419366e-05
9,Вестник Воронежа,4.418355e-05
